# Twitter Sentiment Analysis

Name: Faris Chaudhry

Country: United Kingdom

Email: faris.chaudhry@outlook.com

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.models import KeyedVector
from gensim.scripts.glove2word2vec import glove2word2vec

from textblob import TextBlob
from textblob import Word
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

## Training Data

In [ ]:
traindata_filepath = "train_tweets.csv"
testdata_filepath = "test_tweets.csv"

train_df = pd.read_csv(traindata_filepath)
train_df